# LightGBM v1

### Imports

In [49]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

### Funciones

In [50]:
def normalizar_texto(descripcion):
    
    descripcion = descripcion.replace(',', ' ')
    descripcion = descripcion.replace('.', ' ')
    descripcion = descripcion.replace('<', ' ')
    descripcion = descripcion.replace('>', ' ')
    descripcion = descripcion.replace('\n', ' ')
    descripcion = descripcion.replace('\strong', ' ')
    descripcion = descripcion.replace('\bold', ' ')
    descripcion = descripcion.lower()
    
    return descripcion

def contar_palabras(titulo, palabras):
    
    contadas = 0
    
    for palabra in palabras:
        spliteado = titulo.split()
        contadas += spliteado.count(palabra)
        
    return contadas

### Reads
train.csv -> train
<br>
test.csv -> test

In [102]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

### Preparo el modelo 

**Feature Direción:**

In [103]:
DIRINVALIDAS = set(['Calle','domicilio conocido','calle','CALLE','--','Sin nombre ','1','.','-','x #x','0','...','S/N'])

df = train

df.loc[df.direccion.isin(DIRINVALIDAS), 'direccion'] = 0
df.loc[df['direccion'] != 0, 'direccion'] = 1

df['direccion'].value_counts()

1    237759
0      2241
Name: direccion, dtype: int64

**Feature Titulo y Descripcion:**

In [104]:
df['titulo'] = df['titulo'].fillna(value = ".")
df['titulo'] = df['titulo'].apply(normalizar_texto)
df['descripcion'] = df['descripcion'].fillna(value = ".")
df['descripcion'] = df['descripcion'].apply(normalizar_texto)

df['calefaccion'] = df['titulo'].apply(contar_palabras, palabras=["calefaccion", "calefacción", "calefaccionado", "aireacondicinado",
                    "acondicionado", "estufa","chimenea"]) + df['descripcion'].apply(contar_palabras, palabras=["calefaccion", 
                    "calefacción", "calefaccionado", "aireacondicinado", "acondicionado", "estufa","chimenea"])

df['suite'] = df['titulo'].apply(contar_palabras, palabras=["suite"])+df['descripcion'].apply(contar_palabras, palabras=["suite"])

df['avenida'] = df['titulo'].apply(contar_palabras, palabras=["avenida", "av", "avenidas", "bulevar", 
               "boulevard", "paseo", "vía"])+df['descripcion'].apply(contar_palabras, palabras=["avenida", "av", 
                "avenidas", "bulevar","boulevard", "paseo", "vía"])

df['gim'] = df['titulo'].apply(contar_palabras, palabras=["gimnasio", "gimnásio", "entrenamiento", "gim", "gym", "fit",
            "ejercicio", "gimnasia","atletismo", "cancha"])+df['descripcion'].apply(contar_palabras, palabras=["gimnasio",
            "gimnásio", "entrenamiento", "gim", "gym", "fit","ejercicio", "gimnasia","atletismo", "cancha"])

df['cochera'] = df['titulo'].apply(contar_palabras, palabras=["cochera", "cocheras", "garage", 
                "garages", "garaje", "garajes"])+df['descripcion'].apply(contar_palabras, palabras=["cochera", 
                "cocheras", "garage","garages", "garaje", "garajes"])

df['ubicacion'] = df['titulo'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central"])+df['descripcion'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "zentrica", "céntrico", "céntrica","central"])

df['balcon'] = df['titulo'].apply(contar_palabras, palabras=["balcon", "balcón", "terraza", "palco", "mirador", 
            "balconcillo","azotea", "solana"])+df['descripcion'].apply(contar_palabras, palabras=["balcon", "balcón", 
            "terraza", "palco", "mirador", "balconcillo","azotea", "solana"])

df['camaraseg'] = df['titulo'].apply(contar_palabras, palabras=["camara", "cámara", "cámaras", "camaras","seguridad",
                "guardia","seguro","protegido"]) + df['descripcion'].apply(contar_palabras, palabras=["camara", "cámara",
                "cámaras", "camaras","seguridad","guardia","seguro","protegido"])

df['parque'] = df['titulo'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", "glorieta","jardin",
            "jardín","patio"])+df['descripcion'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", 
            "glorieta","jardin","jardín","patio"])

df['amoblado'] = df['titulo'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble","decorado",
                "listo"]) + df['descripcion'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble",
                "decorado","listo"])

df['bañera'] = df['titulo'].apply(contar_palabras, palabras=["bañera", "hidromasaje", "hidro", "tina",
            "jacuzzi","jacuzi","yacuzi"]) + df['descripcion'].apply(contar_palabras, palabras=["bañera", "hidromasaje", 
            "hidro", "tina","jacuzzi","jacuzi","yacuzi"])


df['estreno'] = df['titulo'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"]) + df['descripcion'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
            "innovador"])

df['transporte'] = df['titulo'].apply(contar_palabras, palabras=["subte", "subterraneo", "subterráneo","metro", 
                "estacion", "estación", "tren","subestacion", "subestación", "ferrocarril","metrobús", "metrobus", 
                "trolebus","trolebús", "bus", "bús"]) + df['descripcion'].apply(contar_palabras, palabras=["subte", 
                "subterraneo", "subterráneo","metro", "estacion", "estación", "tren","subestacion", "subestación", 
                "ferrocarril","metrobús", "metrobus", "trolebus","trolebús", "bus", "bús"])

df['pileta'] = df['titulo'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])+ df['descripcion'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])
                                       
df['lujo'] = df['titulo'].apply(contar_palabras, palabras=["lujo", "delujo", "deluxe", "delúxe", "lujosa", "lujoso", 
            "lujosas", "lujosos","exclusivo","vip"]) + df['descripcion'].apply(contar_palabras, palabras=["lujo", 
            "delujo","deluxe", "delúxe", "lujosa", "lujoso", "lujosas", "lujosos","exclusivo","vip"])

df['humilde'] = df['titulo'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica","credito","crédito","oferta","oferton",
            "imperdible"]) + df['descripcion'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica",
            "credito","crédito","oferta","oferton","imperdible"]) 

df['ventana'] = df['titulo'].apply(contar_palabras, palabras=["ventana", "ventanas", 
            "vista", "ventanal","vistas","cristal"]) + df['descripcion'].apply(contar_palabras, palabras=["ventana",
            "ventanas", "vista", "ventanal","vistas","cristal"])

df['nuevo'] = df['titulo'].apply(contar_palabras, palabras=["reciente", "recien", "recién", "nueva", "nuevo", "nuevas", 
            "nuevos", "estrenar"]) + df['descripcion'].apply(contar_palabras, palabras=["reciente", "recien", "recién", 
            "nueva", "nuevo", "nuevas", "nuevos", "estrenar"])

df['luz'] = df['titulo'].apply(contar_palabras, palabras=["luz", "luminoso", "luminosa","claridad", "luminiscencia", 
        "luminosidad", "iluminación","iluminacion"]) + df['descripcion'].apply(contar_palabras, palabras=["luz", 
        "luminoso", "luminosa","claridad", "luminiscencia","luminosidad", "iluminación","iluminacion"])

df['bueno'] = df['titulo'].apply(contar_palabras, palabras=["bueno", "buena", "buenas", "buenos","excelente", 
            "excelentes","increible","espectacular"]) + df['descripcion'].apply(contar_palabras, palabras=["bueno", 
            "buena", "buenas", "buenos","excelente", "excelentes","increible","espectacular"])

df['contable'] = df['titulo'].apply(contar_palabras, palabras=["precio"]) + df['descripcion'].apply(contar_palabras, palabras=["precio"])

#Nuevos Feat

df['agente'] = df['descripcion'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"]) \
             + df['titulo'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"])

df['garante'] = df['descripcion'].apply(contar_palabras, palabras=["garante", "garantía", "fiador", "garantizador", "avalista", "garantia",
                "defensor", "garantías", "garantes", "codeudor"]) + df['titulo'].apply(contar_palabras, palabras=["garante", 
                "garantía", "fiador", "garantizador", "avalista", "garantia", "defensor", "garantías", "garantes", "codeudor"])

df['finanza'] =  df['descripcion'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario"\
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocréditp", "ahorro"]) + \
                 df['titulo'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario" \
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocrédito", "ahorro"]) 

df['turismo'] =  df['descripcion'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])\
                 + df['titulo'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])

df['longitud_desc'] =  df['descripcion'].apply(lambda x: len(x.split()))

#Nuevas de V4

df['longitud_titulo'] = df['titulo'].apply(lambda x: len(x))

df['cant_!'] = df['titulo'].apply(lambda x: x.count('!')) 

df['tranquilidad'] = df['descripcion'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])\
                 + df['titulo'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])

df['reparacion'] = df['descripcion'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada",
                                                                     "refaccion","refacción","refacciones"])\
                 + df['titulo'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada"
                                                                     "refaccion","refacción","refacciones"])

df['mascotas'] = df['descripcion'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])\
                 + df['titulo'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])

df['accesibilidad'] = df['descripcion'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])\
                 + df['titulo'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])

df['normas'] =  df['descripcion'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])\
                 + df['titulo'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])

df['beneficios'] = df['descripcion'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])\
                 + df['titulo'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])

df['conexion'] = df['descripcion'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])\
                 + df['titulo'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])

df['servicios_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["servicio", "servicios"])

df['metros_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["metros","m2"])

df['acabados'] = df['descripcion'].apply(contar_palabras, palabras = ["acabados","acabado","terminacion","terminación"])

df['plusvalia'] = df['descripcion'].apply(contar_palabras, palabras = ["plusvalia", "plusvalía"])

#Nuevos de V5

df['cocina'] = df['descripcion'].apply(contar_palabras, palabras=['cocina', 'cocinas'])

df['alberca'] = df['descripcion'].apply(contar_palabras, palabras=['alberca'])

df['negacion'] = df['descripcion'].apply(contar_palabras, palabras = ["no"])

df['variospisos'] = df['descripcion'].apply(contar_palabras, palabras = ["escalera","escaleras",
                    "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])

df['vestidor'] = df['descripcion'].apply(contar_palabras, palabras = ["vestidor"])

**Feature Metros:**

In [105]:
df['metrosdescubiertos'] = abs(df['metrostotales']-df['metroscubiertos'])
df['relacionmetros'] = df['metrostotales']/df['metroscubiertos']

In [106]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35)
    return x_train, x_test, y_train, y_test

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

def minmax(serie):
    
    return (serie-serie.min())/(serie.max()-serie.min())

def mean_target_encoding(train, nombrefeat, nombretarget):
    
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean)
    train[nombrefeat + "_mean"]=(temp-temp.min())/(temp.max()-temp.min())
    
    return train

def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

def armar_set_2(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'id'], axis = 1)
    
    #Adiciono servicios
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    train['fecha'] = pd.to_datetime(train['fecha']).dt.year
    
    #"Normalizo" la antiguedad
    #train['antiguedad'] = minmax(train['antiguedad'])
    
    #Elimino los residuos
    #train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                       # 'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train
    x_test["precio"] = y_test
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']

    PMM = df1.groupby('idzona').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']
    
    PMM = df1.groupby('ciudad').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metroscubiertos'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metrosdescubiertos'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio") 
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train = mean_target_encoding(x_train_mean, "idzona", "precio")

    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", x_train_mean)
    x_test = mean_target_decoding(x_test, "tipodepropiedad", x_train_mean)
    x_test = mean_target_decoding(x_test, "ciudad", x_train_mean)
    x_test = mean_target_decoding(x_test, "idzona", x_train_mean)
    
    backup = x_train_mean
    
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    x_test.drop(["precio","provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)

    
    return x_train, x_test, y_train, y_test, backup

In [107]:
x_train, x_test, y_train, y_test, backup = armar_set_2(df)

y = y_train.values

x = x_train.values

y_test = y_test.values

x_test = x_test.values

# divido el df 

/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

### Creo LightGBM data containers

In [108]:
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test)


In [109]:
#Lo entreno

params = {
    'objective': 'regression',
    'colsample_bytree': 0.9731668400523877,
    'learning_rate': 0.05,
    'min_child_samples': 180,#171 
    'min_child_weight': 1e-05, #1e-05
    'num_leaves': 35,#41 
    'reg_alpha': 10, #10
    'reg_lambda': 1000, #100
    'subsample': 0.9,#0.5575732396028996
    'max_depth':-1, 
    'random_state':314, 
    'silent':True,
    'max_bin': 800,
    'metric':'mae', 
    'n_jobs':None, 
    'n_estimators':100000
}

model = lightgbm.train(params,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=10000,
                       early_stopping_rounds=500)

[1]	valid_0's l1: 1.55582e+06
Training until validation scores don't improve for 500 rounds
[2]	valid_0's l1: 1.50551e+06
[3]	valid_0's l1: 1.45807e+06
[4]	valid_0's l1: 1.41375e+06
[5]	valid_0's l1: 1.37117e+06
[6]	valid_0's l1: 1.33155e+06
[7]	valid_0's l1: 1.29364e+06
[8]	valid_0's l1: 1.25744e+06
[9]	valid_0's l1: 1.22404e+06
[10]	valid_0's l1: 1.19224e+06
[11]	valid_0's l1: 1.16329e+06
[12]	valid_0's l1: 1.13503e+06
[13]	valid_0's l1: 1.10889e+06
[14]	valid_0's l1: 1.08386e+06
[15]	valid_0's l1: 1.06056e+06
[16]	valid_0's l1: 1.03795e+06
[17]	valid_0's l1: 1.01626e+06
[18]	valid_0's l1: 996405
[19]	valid_0's l1: 977061
[20]	valid_0's l1: 958841
[21]	valid_0's l1: 941513
[22]	valid_0's l1: 925449
[23]	valid_0's l1: 909658
[24]	valid_0's l1: 894737
[25]	valid_0's l1: 881112
[26]	valid_0's l1: 867615
[27]	valid_0's l1: 854590
[28]	valid_0's l1: 842368
[29]	valid_0's l1: 830607
[30]	valid_0's l1: 820034
[31]	valid_0's l1: 809932
[32]	valid_0's l1: 800142
[33]	valid_0's l1: 790510
[34]

[303]	valid_0's l1: 553066
[304]	valid_0's l1: 553019
[305]	valid_0's l1: 552939
[306]	valid_0's l1: 552869
[307]	valid_0's l1: 552738
[308]	valid_0's l1: 552645
[309]	valid_0's l1: 552568
[310]	valid_0's l1: 552403
[311]	valid_0's l1: 552305
[312]	valid_0's l1: 552238
[313]	valid_0's l1: 552165
[314]	valid_0's l1: 552080
[315]	valid_0's l1: 551967
[316]	valid_0's l1: 551878
[317]	valid_0's l1: 551820
[318]	valid_0's l1: 551721
[319]	valid_0's l1: 551652
[320]	valid_0's l1: 551572
[321]	valid_0's l1: 551454
[322]	valid_0's l1: 551380
[323]	valid_0's l1: 551326
[324]	valid_0's l1: 551234
[325]	valid_0's l1: 551121
[326]	valid_0's l1: 551050
[327]	valid_0's l1: 550990
[328]	valid_0's l1: 550895
[329]	valid_0's l1: 550788
[330]	valid_0's l1: 550715
[331]	valid_0's l1: 550635
[332]	valid_0's l1: 550553
[333]	valid_0's l1: 550482
[334]	valid_0's l1: 550363
[335]	valid_0's l1: 550304
[336]	valid_0's l1: 550229
[337]	valid_0's l1: 550163
[338]	valid_0's l1: 550064
[339]	valid_0's l1: 549999
[

[609]	valid_0's l1: 536577
[610]	valid_0's l1: 536563
[611]	valid_0's l1: 536529
[612]	valid_0's l1: 536499
[613]	valid_0's l1: 536475
[614]	valid_0's l1: 536439
[615]	valid_0's l1: 536408
[616]	valid_0's l1: 536377
[617]	valid_0's l1: 536360
[618]	valid_0's l1: 536330
[619]	valid_0's l1: 536316
[620]	valid_0's l1: 536309
[621]	valid_0's l1: 536299
[622]	valid_0's l1: 536246
[623]	valid_0's l1: 536236
[624]	valid_0's l1: 536206
[625]	valid_0's l1: 536183
[626]	valid_0's l1: 536147
[627]	valid_0's l1: 536101
[628]	valid_0's l1: 536093
[629]	valid_0's l1: 536057
[630]	valid_0's l1: 536025
[631]	valid_0's l1: 535985
[632]	valid_0's l1: 535957
[633]	valid_0's l1: 535908
[634]	valid_0's l1: 535885
[635]	valid_0's l1: 535863
[636]	valid_0's l1: 535835
[637]	valid_0's l1: 535820
[638]	valid_0's l1: 535794
[639]	valid_0's l1: 535779
[640]	valid_0's l1: 535761
[641]	valid_0's l1: 535734
[642]	valid_0's l1: 535711
[643]	valid_0's l1: 535688
[644]	valid_0's l1: 535679
[645]	valid_0's l1: 535639
[

[918]	valid_0's l1: 530015
[919]	valid_0's l1: 530003
[920]	valid_0's l1: 529979
[921]	valid_0's l1: 529976
[922]	valid_0's l1: 529942
[923]	valid_0's l1: 529921
[924]	valid_0's l1: 529911
[925]	valid_0's l1: 529902
[926]	valid_0's l1: 529893
[927]	valid_0's l1: 529877
[928]	valid_0's l1: 529864
[929]	valid_0's l1: 529854
[930]	valid_0's l1: 529843
[931]	valid_0's l1: 529834
[932]	valid_0's l1: 529817
[933]	valid_0's l1: 529774
[934]	valid_0's l1: 529741
[935]	valid_0's l1: 529694
[936]	valid_0's l1: 529666
[937]	valid_0's l1: 529633
[938]	valid_0's l1: 529615
[939]	valid_0's l1: 529586
[940]	valid_0's l1: 529534
[941]	valid_0's l1: 529511
[942]	valid_0's l1: 529494
[943]	valid_0's l1: 529485
[944]	valid_0's l1: 529481
[945]	valid_0's l1: 529472
[946]	valid_0's l1: 529471
[947]	valid_0's l1: 529450
[948]	valid_0's l1: 529437
[949]	valid_0's l1: 529422
[950]	valid_0's l1: 529407
[951]	valid_0's l1: 529401
[952]	valid_0's l1: 529382
[953]	valid_0's l1: 529373
[954]	valid_0's l1: 529365
[

[1216]	valid_0's l1: 525700
[1217]	valid_0's l1: 525695
[1218]	valid_0's l1: 525689
[1219]	valid_0's l1: 525660
[1220]	valid_0's l1: 525652
[1221]	valid_0's l1: 525642
[1222]	valid_0's l1: 525631
[1223]	valid_0's l1: 525620
[1224]	valid_0's l1: 525590
[1225]	valid_0's l1: 525576
[1226]	valid_0's l1: 525559
[1227]	valid_0's l1: 525547
[1228]	valid_0's l1: 525542
[1229]	valid_0's l1: 525526
[1230]	valid_0's l1: 525514
[1231]	valid_0's l1: 525496
[1232]	valid_0's l1: 525487
[1233]	valid_0's l1: 525483
[1234]	valid_0's l1: 525478
[1235]	valid_0's l1: 525467
[1236]	valid_0's l1: 525450
[1237]	valid_0's l1: 525434
[1238]	valid_0's l1: 525426
[1239]	valid_0's l1: 525416
[1240]	valid_0's l1: 525410
[1241]	valid_0's l1: 525410
[1242]	valid_0's l1: 525394
[1243]	valid_0's l1: 525377
[1244]	valid_0's l1: 525363
[1245]	valid_0's l1: 525357
[1246]	valid_0's l1: 525312
[1247]	valid_0's l1: 525311
[1248]	valid_0's l1: 525306
[1249]	valid_0's l1: 525298
[1250]	valid_0's l1: 525289
[1251]	valid_0's l1:

...

[15491]	valid_0's l1: 503923
[15492]	valid_0's l1: 503923
[15493]	valid_0's l1: 503922
[15494]	valid_0's l1: 503923
[15495]	valid_0's l1: 503923
[15496]	valid_0's l1: 503924
[15497]	valid_0's l1: 503921
[15498]	valid_0's l1: 503921
[15499]	valid_0's l1: 503920
[15500]	valid_0's l1: 503918
[15501]	valid_0's l1: 503916
[15502]	valid_0's l1: 503914
[15503]	valid_0's l1: 503914
[15504]	valid_0's l1: 503912
[15505]	valid_0's l1: 503914
[15506]	valid_0's l1: 503916
[15507]	valid_0's l1: 503915
[15508]	valid_0's l1: 503913
[15509]	valid_0's l1: 503912
[15510]	valid_0's l1: 503912
[15511]	valid_0's l1: 503910
[15512]	valid_0's l1: 503912
[15513]	valid_0's l1: 503913
[15514]	valid_0's l1: 503914
[15515]	valid_0's l1: 503915
[15516]	valid_0's l1: 503915
[15517]	valid_0's l1: 503917
[15518]	valid_0's l1: 503917
[15519]	valid_0's l1: 503918
[15520]	valid_0's l1: 503918
[15521]	valid_0's l1: 503919
[15522]	valid_0's l1: 503919
[15523]	valid_0's l1: 503920
[15524]	valid_0's l1: 503921
[15525]	valid_

[15777]	valid_0's l1: 503788
[15778]	valid_0's l1: 503788
[15779]	valid_0's l1: 503787
[15780]	valid_0's l1: 503789
[15781]	valid_0's l1: 503789
[15782]	valid_0's l1: 503788
[15783]	valid_0's l1: 503785
[15784]	valid_0's l1: 503784
[15785]	valid_0's l1: 503785
[15786]	valid_0's l1: 503782
[15787]	valid_0's l1: 503779
[15788]	valid_0's l1: 503776
[15789]	valid_0's l1: 503773
[15790]	valid_0's l1: 503775
[15791]	valid_0's l1: 503774
[15792]	valid_0's l1: 503776
[15793]	valid_0's l1: 503777
[15794]	valid_0's l1: 503778
[15795]	valid_0's l1: 503778
[15796]	valid_0's l1: 503779
[15797]	valid_0's l1: 503781
[15798]	valid_0's l1: 503779
[15799]	valid_0's l1: 503778
[15800]	valid_0's l1: 503775
[15801]	valid_0's l1: 503775
[15802]	valid_0's l1: 503771
[15803]	valid_0's l1: 503772
[15804]	valid_0's l1: 503770
[15805]	valid_0's l1: 503772
[15806]	valid_0's l1: 503771
[15807]	valid_0's l1: 503771
[15808]	valid_0's l1: 503775
[15809]	valid_0's l1: 503775
[15810]	valid_0's l1: 503778
[15811]	valid_

[16061]	valid_0's l1: 503756
[16062]	valid_0's l1: 503756
[16063]	valid_0's l1: 503755
[16064]	valid_0's l1: 503756
[16065]	valid_0's l1: 503760
[16066]	valid_0's l1: 503761
[16067]	valid_0's l1: 503761
[16068]	valid_0's l1: 503760
[16069]	valid_0's l1: 503761
[16070]	valid_0's l1: 503758
[16071]	valid_0's l1: 503757
[16072]	valid_0's l1: 503756
[16073]	valid_0's l1: 503756
[16074]	valid_0's l1: 503756
[16075]	valid_0's l1: 503755
[16076]	valid_0's l1: 503756
[16077]	valid_0's l1: 503755
[16078]	valid_0's l1: 503753
[16079]	valid_0's l1: 503755
[16080]	valid_0's l1: 503753
[16081]	valid_0's l1: 503751
[16082]	valid_0's l1: 503748
[16083]	valid_0's l1: 503748
[16084]	valid_0's l1: 503748
[16085]	valid_0's l1: 503748
[16086]	valid_0's l1: 503747
[16087]	valid_0's l1: 503748
[16088]	valid_0's l1: 503749
[16089]	valid_0's l1: 503749
[16090]	valid_0's l1: 503749
[16091]	valid_0's l1: 503749
[16092]	valid_0's l1: 503747
[16093]	valid_0's l1: 503748
[16094]	valid_0's l1: 503747
[16095]	valid_

[16344]	valid_0's l1: 503738
[16345]	valid_0's l1: 503735
[16346]	valid_0's l1: 503735
[16347]	valid_0's l1: 503735
[16348]	valid_0's l1: 503732
[16349]	valid_0's l1: 503732
[16350]	valid_0's l1: 503733
[16351]	valid_0's l1: 503732
[16352]	valid_0's l1: 503733
[16353]	valid_0's l1: 503730
[16354]	valid_0's l1: 503729
[16355]	valid_0's l1: 503732
[16356]	valid_0's l1: 503734
[16357]	valid_0's l1: 503732
[16358]	valid_0's l1: 503733
[16359]	valid_0's l1: 503735
[16360]	valid_0's l1: 503736
[16361]	valid_0's l1: 503734
[16362]	valid_0's l1: 503735
[16363]	valid_0's l1: 503736
[16364]	valid_0's l1: 503737
[16365]	valid_0's l1: 503738
[16366]	valid_0's l1: 503738
[16367]	valid_0's l1: 503739
[16368]	valid_0's l1: 503739
[16369]	valid_0's l1: 503740
[16370]	valid_0's l1: 503738
[16371]	valid_0's l1: 503738
[16372]	valid_0's l1: 503739
[16373]	valid_0's l1: 503739
[16374]	valid_0's l1: 503738
[16375]	valid_0's l1: 503737
[16376]	valid_0's l1: 503734
[16377]	valid_0's l1: 503733
[16378]	valid_

[16630]	valid_0's l1: 503750
[16631]	valid_0's l1: 503752
[16632]	valid_0's l1: 503750
[16633]	valid_0's l1: 503752
[16634]	valid_0's l1: 503754
[16635]	valid_0's l1: 503751
[16636]	valid_0's l1: 503749
[16637]	valid_0's l1: 503750
[16638]	valid_0's l1: 503748
[16639]	valid_0's l1: 503746
[16640]	valid_0's l1: 503747
[16641]	valid_0's l1: 503746
[16642]	valid_0's l1: 503748
[16643]	valid_0's l1: 503746
[16644]	valid_0's l1: 503746
[16645]	valid_0's l1: 503745
[16646]	valid_0's l1: 503743
[16647]	valid_0's l1: 503742
[16648]	valid_0's l1: 503743
[16649]	valid_0's l1: 503742
[16650]	valid_0's l1: 503740
[16651]	valid_0's l1: 503741
[16652]	valid_0's l1: 503741
[16653]	valid_0's l1: 503740
[16654]	valid_0's l1: 503740
[16655]	valid_0's l1: 503739
[16656]	valid_0's l1: 503739
[16657]	valid_0's l1: 503739
[16658]	valid_0's l1: 503736
[16659]	valid_0's l1: 503735
[16660]	valid_0's l1: 503736
[16661]	valid_0's l1: 503737
[16662]	valid_0's l1: 503737
[16663]	valid_0's l1: 503735
[16664]	valid_

In [110]:
prediccion = model.predict(x_test)

In [111]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

503713.8175684573


# gridsearch

In [45]:
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'regression',
          'nthread': None, # Updated from nthread
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

# creo los parametros para grid  
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [1,2],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['regression'],
    'random_state' : [501], 
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

# pongo los parametros 
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt',
          objective = 'regression',
          n_jobs = None, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

#default:
mdl.get_params().keys()

# Armo grid
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=None)
# Enteno
grid.fit(x, y_train)

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_bin=512, max_depth=-1,
                                     min_child_samples=5, min_child_weight=1,
                                     min_split_gain=0.5, n_estimators=100,
                                     n_jobs=None, num_leaves=31,
                                     objective='binary', random_state=None,
                                     reg_alpha=0...
             iid='warn', n_jobs=None,
             param_grid={'boosting_type': ['gbdt'],
                         'colsample_bytree': [0.65, 0.66],
                         'learning_rate': [0.005], 'n_estimators': [1, 2],
                         'num_leaves': [6, 8, 12, 16],
                         'objective': ['r

In [48]:
grid.best_params_

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.65,
 'learning_rate': 0.005,
 'n_estimators': 2,
 'num_leaves': 16,
 'objective': 'regression',
 'random_state': 501,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'subsample': 0.7}

In [46]:
prediccion = grid.predict(x_test)

In [47]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

1600921.908940707


In [94]:
561929.2799609166 100

SyntaxError: invalid syntax (<ipython-input-94-41a49510c087>, line 1)

# Random Search

In [112]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

sp_uniform(loc=0.2, scale=0.8)

In [147]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

param_test ={
    #'colsample_bytree':  [float(x) for x in np.linspace(0,0.5, num =5)],
    'learning_rate': [0.05,0.1,0.3],
    'min_child_samples': [int(x) for x in np.linspace(30,200, num =8)],#171 
    #1e-05
    'num_leaves': [int(x) for x in np.linspace(30,80, num =3)],#41 
    'reg_alpha': [int(x) for x in np.linspace(0,200, num =8)], #10
    'reg_lambda': [int(x) for x in np.linspace(100,2000, num =8)], #100
    #'subsample': [float(x) for x in np.linspace(0,0.6, num =5)],#0.5575732396028996
    'max_depth':[int(x) for x in np.linspace(-1,200, num =8)], 
    'max_bin': [int(x) for x in np.linspace(500,1500, num =10)]
    
}
print (param_test)

{'learning_rate': [0.05, 0.1, 0.3], 'min_child_samples': [30, 54, 78, 102, 127, 151, 175, 200], 'num_leaves': [30, 55, 80], 'reg_alpha': [0, 28, 57, 85, 114, 142, 171, 200], 'reg_lambda': [100, 371, 642, 914, 1185, 1457, 1728, 2000], 'max_depth': [-1, 27, 56, 85, 113, 142, 171, 200], 'max_bin': [500, 611, 722, 833, 944, 1055, 1166, 1277, 1388, 1500]}


In [148]:
#Numero de ptos a probar 
n_HP_points_to_test = 100

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMRegressor(objective= 'regression', n_jobs=None,min_child_weight= 1e-05, random_state=314,metric='mae', 
     
    n_estimators=20000
    )
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)


In [ ]:
gs.fit(x, y_train)
print('Best score with params: {} '.format( gs.best_params_))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
prediccion = gs.predict(x_test)

In [ ]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

In [ ]:
509